# Lab 1: Machine Learning Pipelines and Grid Search

---

How to Use This Notebook

---

**Recommended Setup**

- For the best experience, **run this notebook on [Google Colab](https://colab.research.google.com/)**—especially if your local machine is slow.  

- In Colab, **enable GPU support** by going to:  

  `Runtime > Change runtime type > Hardware accelerator > GPU`

**Homework Tasks**

 - Homework tasks are clearly marked throughout the notebook in the following format:

   > ---

   > <span style="color:red"><b>TASK X</b> - [<i>some text</i>]:</span>

   > ---

   > ```Your code ....```

   > ---

   > *End of Task X.*

 - For each task:

   - **Complete the code** where indicated.

   - **Upload the required results** from each task to **Homework 1 – Code** on [NextIlearn](https://nextilearn.dsv.su.se).

 - Once you've finished all the tasks:

   Submit your **entire completed notebook (including your code!)** to **Homework 1 – Notebook** on [NextIlearn](https://nextilearn.dsv.su.se).

**Important:**  

Your submission will **only be graded if both files** (code + notebook) are uploaded **before the deadline**. Late submissions are **not accepted**, regardless of technical issues like bad internet connection.

---




This lab will cover various ensemble learning methods covered in the lectures, such as boosting, bagging, and stacking, together with the scikit-learn pipeline. We will tackle the fetal health classification problem using random forest and XGBoost for better performances. We will also learn how to integrate preprocessing and training processes into one pipeline to re-use the whole process easier. The objective of this lab is to get a concept of a whole machine learning pipeline from feature engineering to model selection and lead to a final model.

This lab will mainly focus on:
- Learn how to preprocess and oversample imbalanced datasets.
- Learn how to use the basic random forest, Adaboost, and XGBoost.
- Learn how to use grid search (recap on DAMI) and randomized search.
- Learn how to use the scikit-learn pipeline to construct a complete and reusable preprocessing cycle.

Our goal is to find out the best ensemble model for our dataset. We need to split the dataset into two parts: training and validation sets. Then we test parameter options and eventually choose one best model and compare their test performance.

We will use the [fetal health classification](https://www.kaggle.com/andrewmvd/fetal-health-classification) data from Kaggle. The reduction of child mortality is reflected in several of the United Nations' Sustainable Development Goals and is a crucial indicator of human progress. The UN expects that by 2030, countries end preventable deaths of newborns and children under five years of age, with all countries aiming to reduce under‑5 mortality to at least as low as 25 per 1,000 live births.

Parallel to the notion of child mortality is of course maternal mortality, which accounts for 295 000 deaths during and following pregnancy and childbirth (as of 2017). The vast majority of these deaths (94%) occurred in low-resource settings, and most could have been prevented.

In light of what was mentioned above, Cardiotocograms (CTGs) are a simple and cost-accessible option to assess fetal health, allowing healthcare professionals to take action in order to prevent child and maternal mortality. The equipment itself works by sending ultrasound pulses and reading its response, thus shedding light on fetal heart rate (FHR), fetal movements, uterine contractions, and more.

This dataset contains 2126 records of features extracted from Cardiotocogram exams, which were then classified by three expert obstetricians into three classes:

- Normal: 1
- Suspect: 2
- Pathological: 3

We will first try to apply ensemble models and eventually integrate grid search and pipeline to create an automated evaluation process to find the best model.

### 1. Data exploration

Let's locate the dataset anywhere you want in your local and load it using pd.read_csv. The dataset can be downloaded [here](https://www.kaggle.com/andrewmvd/fetal-health-classification). This time, we will return to pandas DataFrame!

In [ ]:
import pandas as pd
import numpy as np
import imblearn # install if you don't have it
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Create a DataFrame
data = pd.read_csv("fetal_health.csv")

In [ ]:
# head
data.head()

**Duplicates**: The first thing we need to do is to separate features and labels. However, before that, let's check if the data has some duplicated rows.

In [ ]:
# Check for duplicates in data
data.duplicated()

In [ ]:
# How many duplicates
data.duplicated().sum()

In [ ]:
# drop_duplicates
data_dropped = data.drop_duplicates()

In [ ]:
# Check dropped datapoints and compare to before dropping duplicates
data_dropped

In [ ]:
data

**Features/labels split**: We can choose the columns from the dataframe and create a new variable X.
- what would be our label?
- how could we only choose the columns?

In [ ]:
# Features
X = data_dropped.iloc[:, :-1]
# Labels
y = data_dropped["fetal_health"].astype(int) # Pandas series shares some functions with NumPy!

In [ ]:
# X.head
X.head()

In [ ]:
# y.head
y.head()

In [ ]:
# describe - Basic statistics
X.describe()

Today, our focus is to learn about ensemble methods, so we would not focus on many data exploration strategies, but let's try to investigate a few things. First, let's check if the labels are balanced or not using seaborn.
- [Useful seaborn functions](https://seaborn.pydata.org/tutorial/function_overview.html)

In [ ]:
import seaborn as sns

In [ ]:
# countplot - check the proportion visually
sns.countplot(x=y)

In [ ]:
# series.value counts - check the proportion
y.value_counts()

In [ ]:
# proportion
y.value_counts() / len(y)

It is natural that the normal cases are most common. It might lead to higher accuracy too, as label 1 keeps about 78% of the whole dataset. However, to make a robust model that does not have any relationship with the distribution the dataset typically has, we may need to consider it so the model should not be biased towards the normal cases.

In [ ]:
# isna - Missing data check
X.isna()

In [ ]:
X.isna().sum()

**Duplicates**: Would there be any rows identical to each other?

In [ ]:
X.duplicated().sum()

Would it be better to remove these conflicting instances? In this case we need to align the labels too.

In [ ]:
X_dropped = X[~X.duplicated()]
y_dropped = y[~X.duplicated()]

In [ ]:
# Correlation check - among the features
mat = X_dropped.corr()
mask = np.triu(np.ones_like(mat, dtype=bool))
fig, ax = plt.subplots(figsize = (15,10))
ax = sns.heatmap(mat, mask=mask, annot = True, linewidths=0.5, fmt = ".2f", square=True)

We may need to apply **1) standardization** and **2) oversampling the minority** for better performance. We might be able to apply those methods one by one, but it will be better to streamline things into one pipeline.

In [ ]:
# Correlation check - features and the label
plt.figure(figsize=(5, 12))
heatmap = sns.heatmap(data.corr()[['fetal_health']].sort_values(by='fetal_health', ascending=True), fmt="0.2f", annot=True)
heatmap.set_title('Features correlating with the label')
heatmap.set_ylim([0,22])

### 2. Preprocessing

**Train test split**: After dividing the dataset into multiple parts, preprocessing like standardization or over-sampling should be applied. If not, we will end up with a cheating model that already knows part of the test information.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create X_train, X_test, y_train, y_test using train_test_split with stratification. Set test size=0.3.
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.3, stratify=y_dropped)

**Standardization**: We can simply apply the standardization function we made in the previous lab or scikit-learn's method to standardize the dataset. This time, let's stick to scikit-learn. You can use scikit-learn's **StandardScaler** to apply standardization (= set the mean and the standard

In [ ]:
# StandardScaler
# X_train_scaled, X_test_scaled
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X=X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = scaler.transform(X=X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

In [ ]:
# describe - standardized mean and standard deviation
X_train_scaled.describe()

In [ ]:
y_train.value_counts()

**Oversampling**: To over-sample the instances from the minority classes (2 and 3), we will use a new library called **imblearn** (which stands for Imbalanced-(scikit)-learn). This library contains various classifiers and resampling methods for imbalanced datasets. Let's use the most straightforward method it has.

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Define RandomOverSampler to a variable ros
# ratio
ros = RandomOverSampler(random_state=12345) # random seed
# Create X_resampled, y_resampled using the fit_resample method or fit + sample.
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_scaled, y_train)

In [ ]:
# value_counts
y_train_resampled.value_counts()

Unfortunately, this simple oversampler just duplicates the instances we already have. So we can regard that it's just a weighting factor putting more weights on the minority classes.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[8, 4])
ax[0].scatter(X_train_resampled.iloc[:, 0], X_train_resampled.iloc[:, 7], c=y_train_resampled, s=20, linewidth=0.5, edgecolor='black')
ax[1].scatter(X_train_scaled.iloc[:, 0], X_train_scaled.iloc[:, 7], c=y_train, s=20, linewidth=0.5, edgecolor='black')
ax[0].set_title("Resampled")
ax[1].set_title("Original")
plt.show()

There are some more advanced techniques to resample such as **SMOTE** or **ADASYN**...

- [What does SMOTE do?](https://www.researchgate.net/publication/287601878/figure/fig1/AS:316826589384744@1452548753581/The-schematic-of-NRSBoundary-SMOTE-algorithm.png)
- [What do the others do?](https://www.kaggle.com/code/residentmario/oversampling-with-smote-and-adasyn)
- [EasyEnsemble and BalanceCascade](https://www.kaggle.com/code/residentmario/undersampling-and-oversampling-imbalanced-data/notebook)
- [SMOTE variants](https://imbalanced-learn.org/stable/auto_examples/over-sampling/plot_comparison_over_sampling.html#sphx-glr-auto-examples-over-sampling-plot-comparison-over-sampling-py)

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN

In [ ]:
# SMOTE, ADASYN
# SMOTE is weak when we have a few outliers that do not contribute the classification performance.
# X_resampled_SMOTE, y_resampled_SMOTE
X_resampled_SMOTE, y_resampled_SMOTE = SMOTE().fit_resample(X_train_scaled, y_train)
# X_resampled_ADASYN, y_resampled_ADASYN
X_resampled_ADASYN, y_resampled_ADASYN = ADASYN().fit_resample(X_train_scaled, y_train)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=[16, 4])
ax[0].scatter(X_train_resampled.iloc[:, 0], X_train_resampled.iloc[:, 7], c=y_train_resampled, s=20, linewidth=0.5, edgecolor='black')
ax[1].scatter(X_resampled_SMOTE.iloc[:, 0], X_resampled_SMOTE.iloc[:, 7], c=y_resampled_SMOTE, s=20, linewidth=0.5, edgecolor='black')
ax[2].scatter(X_resampled_ADASYN.iloc[:, 0], X_resampled_ADASYN.iloc[:, 7], c=y_resampled_ADASYN, s=20, linewidth=0.5, edgecolor='black')
ax[3].scatter(X_train_scaled.iloc[:, 0], X_train_scaled.iloc[:, 7], c=y_train, s=20, linewidth=0.5, edgecolor='black')
ax[0].set_title("Resampled_RANDOM")
ax[1].set_title("Resampled_SMOTE")
ax[2].set_title("Resampled_ADASYN")
ax[3].set_title("Original")

plt.show()

Sometimes there have been some practices to keep the minority classes at most 50% of the majority (so in our case, classes 2 and 3 won't have more than 827 instances).

### 3. Simple ensemble

A simple ensemble model with a hard voting / soft voting can be created using scikit-learn's `VotingClassifier`.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [ ]:
dt = DecisionTreeClassifier()
lr = LogisticRegression()
svm = SVC()

In [ ]:
voting = VotingClassifier([('lr', lr), ('dt', dt), ('svm', svm)], voting='hard')
voting

In [ ]:
# fit the method on X_train_scaled and y_train
voting.fit(X_train_scaled, y_train)

Check the scores of each method and the voting classifier.

In [ ]:
for clf in (dt, lr, svm, voting):
  clf.fit(X_resampled_SMOTE, y_resampled_SMOTE)
  print(clf.__class__.__name__, clf.score(X_test_scaled, y_test))

In [ ]:
for clf in (dt, lr, svm, voting):
  clf.fit(X_resampled_ADASYN, y_resampled_ADASYN)
  print(clf.__class__.__name__, clf.score(X_test_scaled, y_test))

In [ ]:
for clf in (dt, lr, svm, voting):
  clf.fit(X_train_scaled, y_train)
  print(clf.__class__.__name__, clf.score(X_test_scaled, y_test))

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = voting.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

Those are our basic performance scores.

### 4. Practical ensemble methods

We may use some ensemble methods that were covered in the lecture. To just use those models, we can still stick to the standard scikit-learn process. Random forests and AdaBoost are supported by scikit-learn. However, you need a separate library for XGBoost.

#### Bagging

`BaggingClassifier` supports the following modes:
 - For samples: bagging, pasting (without replacement)
 - For features: Random patches (sampling both samples and features), random subspaces (sampling only features)


In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
# max_features, bootstrap_features
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500, bootstrap = True, n_jobs = -1)
bag_clf.fit(X_train_scaled, y_train)
bag_clf.score(X_test_scaled, y_test)

**Out-of-bag evaluation**: Average the scores using the out of bag samples. To use the oob_score, you need to turn it on when you initialize the classifier.

In [ ]:
#oob_score = True
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500, bootstrap = True, n_jobs = -1, oob_score=True)
bag_clf.fit(X_train_scaled, y_train)
bag_clf.score(X_test_scaled, y_test)

In [ ]:
# After training, you can check the score from oob_score_.
bag_clf.oob_score_

In [ ]:
# oob_decision_function_ will show the probabilities for each oob instance
bag_clf.oob_decision_function_

#### Random forests = Decision tree trained via bagging

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

Also check: [ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)

#### AdaBoost (Adaptive Boosting) - Various classifiers / IT CANNOT BE PARALLELIZED

AdaBoost is a normal boosting algorithm, creating multiple weak learners throughout multiple phases - so each phase is dependent on the previous one. Since the weight can be applied to any classifier, you can change the base classifier as you want.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# AdaBoostClasifier
# - base_estimator -> default estimator is Decision Stump
ab = AdaBoostClassifier()
ab.fit(X_train_scaled, y_train)
print(ab.score(X_test_scaled, y_test))

In [ ]:
# - base_estimator = LogisticRegression()
ab = AdaBoostClassifier(estimator=LogisticRegression())
ab.fit(X_train_scaled, y_train)
print(ab.score(X_test_scaled, y_test))

#### XGBoost (Gradient Boosting) - IT CAN BE PARALLELIZED

XGBoost is a decision-tree-based gradient boosting classifier with more advanced tricks inside to make itself a more scalable and better performance. The key differences are as follows:

 - An improved data structure for better utilization such as sparse matrices
 - Better support for multicore processing.
 - L1/L2 regularization
 - Tree-node dropout


 To best exploit XGBoost, you need to perform [parameter optimization](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/).

In [ ]:
# - XGBoost
import xgboost as xgb

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Then train the model
xgbm = xgb.XGBClassifier()
xgbm.fit(X_train_scaled, y_train_encoded)
print(xgbm.score(X_test_scaled, y_test_encoded))

XGBoost also supports to handle imbalanced datasets by putting different weights based on the class distribution. For more details, [check this article](https://machinelearningmastery.com/xgboost-for-imbalanced-classification/).

 - scale_pos_weight with GridSearchCV


### 5. Grid search and Randomized search

#### Grid search

Grid search is a method of tuning the optimal hyperparameter value by trying all possible combinations of the parameters of interest, provided by scikit-learn of python. Grid search has the disadvantage of being very expensive and time-consuming to calculate because it evaluates all combinations.

#### Randomized search

Randomized search finds optimal hyperparameter values by generating random numbers within a specified range. Hyperparameters are of different relative importance, and random searches are advantageous for optimization because more important parameters can be explored. In fact, there are also indicators that random searches in papers yield better results with fewer search opportunities. Random Search can stop searching at any time. This is because even if you stop in the middle, the search is not biased towards a specific range.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Our parameter distribution (or parameter grid in the case of grid search)
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

Compared to the grid search, the randomized search has some randomness and iterations, so it has a few more parameters such as **n_iter**, **random_state**. Also, the parameter grid is now called parameter distributions as it can also receive a distribution (such as **scipy.stats.uniform**) and sample from it.

In [ ]:
# RandomizedSearchCV - create the randomized search instance with those parameters
# - estimator: random forest
# - param_distribution: random_grid
# - n_iter: 100
# - cv: 3

# We also have those optional parameters
# - random_state
# - refit (default = True)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)

In [ ]:
%%time
rf_random.fit(X_train_scaled, y_train)

In [ ]:
# best_estimator
rf_random.best_estimator_

In [ ]:
# best_score
rf_random.best_score_

In [ ]:
# test score
rf_random.score(X_test_scaled, y_test)

You can also run a grid search and see if it completes in a reasonable amount of time.

In [ ]:
gs_random = GridSearchCV(estimator = rf, param_grid = random_grid, cv = 3, verbose=2, n_jobs = -1)

In [ ]:
%%time
gs_random.fit(X_train_scaled, y_train) # NEVER FINISHES!!!!!

Randomized search is greatly helpful when we have many options to search as it randomly choose the combination. However, it does not guarantee that it will find the optimal solution, but still it will give you similar result with the grid search. However, if you only have a limited number of parameter combinations, it is still the best to apply the grid search.

### 6. Pipelines

Most machine learning data is not ideal for building a final model. Many data transformations such as manipulating categorical variables or scaling and normalization must be performed. Scikit-Learn has built-in most commonly used functions for preprocessing functions. However, in a typical machine learning workflow, you would need to apply all these transformations more than once. Because you do it once when you train the model and again for every new data you want to predict. Of course, you can use a reusable function, but you will have to run it first and then call the model separately. The Scikit-Learn pipeline is a tool that simplifies this process and has several key advantages:

- Pipelines allow you to create a single object that contains all stages of data preprocessing and classification.
- It makes cross-validation and other types of model selection easier.
- Increased reproducibility

Using pipeline lets you manage your whole preprocessing and training process in one place and change the setting in a central place.
- [Advanced pipeline tutorial and parameter tuning](https://towardsdatascience.com/advanced-pipelines-with-scikit-learn-4204bb71019b)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Create a pipeline
# - featureGenerator = PolynomialFeatures
# - scaler = StandardScaler
# - selector = VarianceThreshold with 0.1
# - classifier = RandomForestClassifier
# etc: LabelEncoder ... OneHotEncoder ... KBinsDiscretizer ... SimpleImputer

pipe = Pipeline([
  ('featureGenerator', PolynomialFeatures()),
  ('scaler', StandardScaler()),
  ('selector', VarianceThreshold(0.1)),
  ('classifier', RandomForestClassifier())
])





We can fit the pipeline in the same way as general scikit-learn classifiers.

In [ ]:
pipe.fit(X_train_scaled, y_train)

In [ ]:
print('Training set score: ' + str(pipe.score(X_train_scaled, y_train)))
print('Test set score: ' + str(pipe.score(X_test_scaled, y_test)))

---

<span style="color:red"><b>TASK 1</b> - Add private functions to pipelines:</span>

---
Most of the functions are supported by scikit-learn, but sometimes you may need other methods as well to change the dataset. In this case, you may have some modules that are not part of the pipeline. However, the scikit-learn pipeline also supports custom functions following some rules.

The rule here is to construct a class with both fit and transform methods inside.



- Create a class `SquaredFeatureTransformer` that squares the values in a numpy array and merges those squared features into the original array (i.e., the number of features is doubled). Note that we assume the dataset is in a tabular format, which means columns would be features, while rows would be instances.
- Our dataset will have two times more features after this transformation.
- Integrate this transformer into the pipeline, by adding the class `SquaredFeatureTransformer` with the name `squared`. The whole pipeline structure should be as follows:
  - scaler = StandardScaler
  - squared = SquaredFeatureTransformer
  - selector = VarianceThreshold with 0.1 variance threshold
  - classifier = RandomForestClassifier
- Eventually, the pipeline should have four elements: `scaler`, `squared`, `selector`, and `classifier`. Define this pipeline to the variable pipe.
- Understand how to create a custom transformer function.

Please note that in the grading system on nextilearn, all necessary libraries are already imported. Do not import any libraries.

In [ ]:
class SquaredFeatureTransformer(BaseEstimator, TransformerMixin):
  #This class will transform the dataset by adding squared features
  #You need to change the transform function to square each feature's values and add them to the matrix.

  def __init__(self):
    return None

  def fit(self, X, y = None):
    return None

  def transform(self, X, y = None):
    return None

pipe = None # CHANGE IT

---

<span style="color:red"><b>TASK 2</b> - Integrate grid search into pipelines:</span>

---
So far, we have learned randomized search and grid search for a parameter search of a specific model and the scikit-learn pipeline to streamline the complete machine learning task from preprocessing to model training. Then the remaining question is how we can also integrate the model selection process - like the parameter search we performed using the grid search and the randomized search - into the pipeline.

By adding those parameter tests, you can also change the modules that should be in the same name: for example, you can have `[StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler()]` in a name 'scaler' of the pipeline. Moreover, you can perform a grid test if they have the common parameters - they usually have.

Define a new variable parameters and customize the options by creating a dictionary. You can change the pipeline values by setting the same key as you did for the pipeline. For example, if you would like to test two scalers, you can put 'scaler': [scaler1, scaler2].

Create the following key-values in the parameters dictionary.

 - scaler: `[StandardScaler(), MinMaxScaler()]`.
 - classifier: `[AdaBoostClassifier(), RandomForestClassifier()]`.

You can also search for many parameter options by using underscores. Please check this guideline. If you want to test the threshold parameter of the selector (here, we use `VarianceThreshold` as a selector).

Your parameter grid should also have a threshold for `VarianceThreshold` with the values `[0, 0.001, 0.01]`.

Define a grid search with cross validation into the variable grid_search. You can put the predefined pipeline instance pipe as a classifier and should use your parameter grid. The search should run 5-fold cross validation.

In [ ]:
# Define the parameter grid
parameters = None # CHANGE IT

# Run grid search
grid_search = None # CHANGE IT

---

<span style="color:red"><b>TASK 3</b> - Nested k-fold cross validation:</span>

---

In this exercise, you will implement nested k-fold cross-validation using Scikit-Learn to evaluate and compare different machine learning models while tuning their hyperparameters. The goal is to perform model selection in a robust and unbiased manner, ensuring that hyperparameter tuning and model evaluation are conducted on separate data splits. You will work with the breast_cancer dataset from sklearn.datasets, where the objective is to classify whether a tumor is malignant or benign. Please check this guideline to see how to open the dataset.

You will define a pipeline that includes a scaler (`StandardScaler`), a feature selector (`VarianceThreshold`), and a classifier. Two classifiers will be tested: `RandomForestClassifier` and `AdaBoostClassifier`. Each classifier will have a specific set of hyperparameters to tune:

- **RandomForestClassifier parameters**:
  - `n_estimators: [50, 100]`
  - `max_depth: [None, 10]`
- **AdaBoostClassifier parameters**:
  - `n_estimators: [50, 100]`
  - `learning_rate: [0.01, 0.1]`

You may create a dictionary called `param_grids` having two keys `'RandomForest', 'AdaBoost'` where each has a parameter grid for each classifier as a value. Then you will apply nested cross-validation with:

Outer loop (2-fold stratified cross-validation): Splits the dataset into 2 folds, where 1 fold is used as the test set and the remaining 4 folds are used for training. For this instance, set the random state to 42 and shuffle the dataset.
Inner loop (2-fold stratified cross-validation using `GridSearchCV`): Finds the best hyperparameters for each classifier on the training set of the outer loop. For this instance, set the random state to 42 and shuffle the dataset.

For each outer loop iteration, fit the best model (determined by `GridSearchCV`) on the training data and evaluate it on the outer test set. At the end, compute the mean accuracy across all outer folds to compare the models. The classifier with the highest mean accuracy across the outer folds is considered the best model for this dataset. Save the final accuracy value into the variable final_accuracy in the format of dictionary with two keys: `'RandomForest', 'AdaBoost'`. For validation, make sure to set the random state of `RandomForestClassifier` and `AdaBoostClassifier` to 42.

Note that while we provide you with skeleton code, you are free to implement in your own preferred way. We will only validate the `final_score` dictionary if the average performance is correct.